In [72]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
import numpy as np

from keras.callbacks import EarlyStopping

In [73]:
# parameter transfer learning
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

In [74]:
x = base_model.output # untuk mendapatkan output dari model VGG16
x = GlobalAveragePooling2D()(x) # mengubah fitur dari layer konvolusional menjadi vektor tunggal
x = Dense(1024, activation='relu')(x) # mempelajari pola-pola kompleks dari fitur-fitur yang diekstraksi oleh layer konvolusional
predictions = Dense(4, activation='softmax')(x) # untuk menghasilkan probabilitas untu setiap kelas
model = Model(inputs=base_model.input, outputs=predictions) # mendefinisikan input dari model VGG16 dan output dari prediksi

In [75]:
#untuk menjaga bobot dari model dasar tetap stabil dan fokus melatih bagian yang baru saja
for layer in base_model.layers:
    layer.trainable = False

In [76]:
import os
import shutil
import random

# Direktori asal gambar dan direktori tujuan train dan validation
original_dataset_dir = 'E:\\SKRIPSI_MIRZA\\batik'
base_dir = 'E:\\SKRIPSI_MIRZA\\data'

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

os.makedirs(train_dir, exist_ok=True)
os.makedirs(validation_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

for class_name in ['kawung', 'megamendung','parang','random']:  # nama kelas
    os.makedirs(os.path.join(test_dir, class_name), exist_ok=True)
    os.makedirs(os.path.join(train_dir, class_name), exist_ok=True)
    os.makedirs(os.path.join(validation_dir, class_name), exist_ok=True)

    files = os.listdir(os.path.join(original_dataset_dir, class_name))
    random.shuffle(files)
    
    train_split = int(0.7 * len(files))
    validation_split = int(0.9 * len(files))

    train_files = files[:train_split]
    validation_files = files[train_split:validation_split]
    test_files = files[validation_split:]
    
    for file in train_files:
        shutil.copy(os.path.join(original_dataset_dir, class_name, file),
                    os.path.join(train_dir, class_name, file))
    
    for file in validation_files:
        shutil.copy(os.path.join(original_dataset_dir, class_name, file),
                    os.path.join(validation_dir, class_name, file))

    for file in test_files:
        shutil.copy(os.path.join(original_dataset_dir, class_name, file),
                    os.path.join(test_dir, class_name, file))

In [77]:
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical')

Found 194 images belonging to 4 classes.
Found 103 images belonging to 4 classes.
Found 64 images belonging to 4 classes.


In [78]:
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [79]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [80]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.n // train_generator.batch_size,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_generator.n // validation_generator.batch_size,
    callbacks=[early_stopping]
)

Epoch 1/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 16s 3s/step - accuracy: 0.3019 - loss: 1.4492 - val_accuracy: 0.6667 - val_loss: 0.8765
Epoch 2/10
1/6 ━━━━━━━━━━━━━━━━━━━━ 8s 2s/step - accuracy: 0.6250 - loss: 1.0277

c:\Users\BAYU\AppData\Local\Programs\Python\Python311\Lib\contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 97ms/step - accuracy: 0.6250 - loss: 1.0277 - val_accuracy: 0.8571 - val_loss: 0.7809
Epoch 3/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 15s 3s/step - accuracy: 0.6868 - loss: 0.8298 - val_accuracy: 0.7604 - val_loss: 0.6195
Epoch 4/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 88ms/step - accuracy: 0.8125 - loss: 0.6075 - val_accuracy: 0.8571 - val_loss: 0.6551
Epoch 5/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 16s 3s/step - accuracy: 0.8067 - loss: 0.5683 - val_accuracy: 0.8229 - val_loss: 0.5037
Epoch 6/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 97ms/step - accuracy: 0.8750 - loss: 0.3915 - val_accuracy: 1.0000 - val_loss: 0.3032
Epoch 7/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 14s 3s/step - accuracy: 0.8099 - loss: 0.4884 - val_accuracy: 0.8646 - val_loss: 0.3567
Epoch 8/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 78ms/step - accuracy: 0.8125 - loss: 0.4239 - val_accuracy: 1.0000 - val_loss: 0.1820
Epoch 9/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 15s 3s/step - accuracy: 0.8648 - loss: 0.3754 - val_accuracy: 0.8542 - val_loss: 0.3882
Epoch 10/10
6/6

In [81]:
for layer in base_model.layers[:15]:
    layer.trainable = False
for layer in base_model.layers[15:]:
    layer.trainable = True

In [87]:
from tensorflow.keras.preprocessing import image
import numpy as np

# Nama kelas yang ada dalam model Anda
class_names = ['kawung', 'megamendung', 'parang','random']

def preprocess_image(image_path):
    # Muat dan preprocess gambar
    img = image.load_img(image_path, target_size=(150, 150))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0  # Normalisasi
    return img_array

def predict_image(image_path, model, threshold=0.8):
    img_array = preprocess_image(image_path)
    predictions = model.predict(img_array)
    max_prob = np.max(predictions)
    predicted_class = np.argmax(predictions)

    if max_prob < threshold:
        return 'random'
    elif predicted_class == 0:
        return 0  # Kawung
    elif predicted_class == 1:
        return 1  # Megamendung
    elif predicted_class == 2:
        return 2  # Parang
    else:
        return'random'

# Contoh penggunaan
image_path = 'E:\\latihan\\my_flask_app\\static\\css\\images\\12.jpg'  # Ganti dengan path gambar sembarang Anda
result = predict_image(image_path, model, threshold=0.8)
print(f'Predicted Class: {result}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step
Predicted Class: random


In [85]:
model.save('agus_babik.keras')